In [1]:
#imports
import math
import numpy as np
import random
import itertools
from random import choices
import cmath
BOLD = '\033[1m'
RESET = '\033[0m'


In [2]:
#generating population in dbm
def generate_random_solutions(powers, no_of_users, num_random_solutions):
    return np.random.choice(powers, size=(num_random_solutions, no_of_users))


In [4]:
# converting population to linear watt
def calculate_linear_trans_power(pop_size_dbm):
    result=(pow(10, -3) * np.power(10.0, np.array(pop_size_dbm, dtype="float") / 10))
    return result


In [8]:
# fitness function  ==== linear_power_vect
def sum_linear_powers(array):
    return np.sum(array, axis=1)

In [11]:
#usage for converting the noise to linear
D=10**6  #bandwidth
N=-174 + 10 * math.log10(D) #noise in dbm
linear_noise = calculate_linear_trans_power([N])

In [12]:
#calculate the bit rate matrix
def calculate_formula_and_sums(linear_pop, SUM_GAIN, linear_noise):
    a = np.array(linear_pop)
    b = np.array(SUM_GAIN)
    bit_rate_matrix = (D * np.log2(1 + ((a * b) / linear_noise)))
    return bit_rate_matrix

In [19]:
def selection_half_population(size_pop,DBM_pop,linear_power_vect):

 
    sorted_help_fun=sorted(help_fun, key=lambda help_fun: help_fun[1],reverse=False)
    
    
    parent    = sorted_help_fun[:L,0]
    parent_EE = sorted_help_fun[:L,1]

 

    return  parent_DMB,parent_power_LIN

SyntaxError: invalid non-printable character U+00A0 (3694592784.py, line 3)

In [20]:
help_fun = [[[0,0,2],5],[[6,0,2],56],[[0,8,2],2]]

sorted_help_fun=sorted(help_fun, key=lambda help_fun: help_fun[1],reverse=False)

sorted_help_fun

[[[0, 8, 2], 2], [[0, 0, 2], 5], [[6, 0, 2], 56]]

In [12]:
# bit rate vector
def row_wise_sums(matrix):
    return [sum(row) for row in matrix]

In [13]:
#sorting linear power
def sort_row_sums(array):
    return np.sort(array)


In [15]:
# selecting the half of population
def select_first_half(array):
    midpoint = len(array) // 2
    return array[:midpoint]


In [17]:
def get_top_half_dbm(pop_size_dbm, row_sums, first_half_linear_power_increasing):
    # Get the indices of the top half of the sorted row sums of linear power
    top_half_linear_power_indices = [np.where(row_sums == value)[0][0] for value in first_half_linear_power_increasing]

    # Get the power allocations in dBm corresponding to the top half of the sorted row sums of linear power
    top_half_linear_power_dbm = pop_size_dbm[top_half_linear_power_indices]

    return top_half_linear_power_dbm


In [ ]:
pop_size_dbm = 4
row_sums     = 

get_top_half_dbm(pop_size_dbm, row_sums, first_half_linear_power_increasing)

In [ ]:
# Usage
top_half_dbm = get_top_half_dbm(pop_size_dbm, row_sums, first_half_linear_power_increasing)
print(f"{BOLD}TOP HALF OF POWER ALLOCATIONS IN DBM CORRESPONDING TO SORTED ROW SUMS OF LINEAR POWER(INITIAL POPULATION){RESET}")
print(top_half_dbm)


In [ ]:
def simple_single_point_crossover(population):
    num_offspring = len(population)
    offspring = []

    for i in range(0, num_offspring, 2):
        # Select two parent solutions sequentially
        parent1, parent2 = population[i], population[i + 1]

        # Randomly choose a crossover point
        crossover_point = random.randint(1, len(parent1) - 1)

        # Create new offspring by swapping the genes of the parents at the crossover point
        offspring1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
        offspring2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))

        offspring.extend([offspring1, offspring2])

    return np.array(offspring)

# Usage
offspring = simple_single_point_crossover(top_half_dbm)

print(f"{BOLD}OFFSPRING AFTER SINGLE-POINT CROSSOVER{RESET}")
print(offspring)


In [ ]:
def mutation(offspring, powers, mutation_rate=1.0):
    mutated_offspring = offspring.copy()

    for i in range(len(offspring)):
        if random.random() < mutation_rate:
            # Randomly select a gene index for mutation
            gene_index = random.randint(0, len(offspring[i]) - 1)

            # Choose a new power value randomly, different from the current gene value
            new_power = random.choice([p for p in powers if p != offspring[i][gene_index]])

            # Replace the gene with the new power value
            mutated_offspring[i][gene_index] = new_power

    return mutated_offspring



In [ ]:
# Usage
mutated_offspring = mutation(offspring, powers)
print(f"{BOLD}MUTATED OFFSPRING{RESET}")
print(mutated_offspring)

In [ ]:
def combine_initial_and_mutated(initial_population, mutated_offspring):
    return np.concatenate((initial_population, mutated_offspring), axis=0)

# Usage
combined_population = combine_initial_and_mutated(top_half_dbm, mutated_offspring)
print(f"{BOLD}COMBINED POPULATION{RESET}")
print(combined_population)
